# IBM Professional Certificate in Data Science Final Project
My family is looking for a home to buy.  This project was created to fulfill the final project requirements for IBM's Data Science certificate program and to aid my family in that search.  First, I will need data on houses for sale.  I will scrape that data from Trulia.com, a realty aggregator.  Next, I want to score houses based on several factors to find which homes best fulfill criteria important to my family.  

Using the Trulia data, I will calculate square feet per dollar, since purchasing more home for the same price is desirable. We would also like to live close to my parents and siblings, so we will be limiting our search geographically, then calculating exact distances to each family member.  **Note: since this project is being shared with strangers, I am using arbitrary addresses rather than their actual addresses for privacy reasons.**  Next, I will calculate the distance to the nearest Costco, as this is our favorite store and represents a significant portion of our annual grocery and home-goods expenses.  This will be done using Foursquare's API to discover nearby Costco locations.  Additionally, I will use the Foursquare API to generate information on the variety of nearby venues, as we would like plenty of options for dining, entertainment, etc.

Finally, I will use these variables to calculate a desirability score for each home, printing a table in order of desirability and visualizing the homes on a map using folium.

# We begin by importing the necessary libraries.

In [1]:
#Install and import the required libraries.  If necessary, pip install statements can be used to install libraries first.
import sys
from bs4 import BeautifulSoup as BS #BeautifulSoup is a Python library
                                    #for pulling data out of HTML and XML files.
import urllib.request
from urllib.request import urlopen,Request
import urllib.parse
import urllib.error
import ssl
import re
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr

### Now we will scrape data on houses for sale from Trulia.com.  We will use some filters built in to Trulia's website to pre-filter our results.

In [2]:
def get_headers():
    #Headers
    headers={'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'accept-language':'en-US,en;q=0.9',
            'cache-control':'max-age=0',
            'upgrade-insecure-requests':'1',
            'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'}

    return headers


ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE
count=1 # for pagination
#Create variables for editing URL filter criteria.
num_beds = '3p_beds' #minimum of 3 bedrooms in the home.
num_baths = '2p_baths' #minimum of 2 bathrooms in the home.
price_range = '100000-320000_price' #Set minimum and maximum price for the home.
min_size = '1750p_sqft' #set minimum size of home to 1750 sqft
house_type = 'SINGLE-FAMILY_HOME_type' #Only interested in single-family homes.
max_age = '1979p_built' #Built after asbestos outlawed.
sale_type = 'fsbo,resale_lt/' #This prevents new construction/buildable plans from appearing.

#Instatiate lists for holding scraped data.
price=[]
add1=[]
area=[]
bed=[]
bath=[]
floor=[]

#List of starting URLs for beautiful soup to pull data from.
urls = ["https://www.trulia.com/for_sale/Lafayette,IN/{}/{}/{}/{}/{}/{}/{}".format(num_beds,num_baths,price_range,min_size,house_type,max_age,sale_type),
        "https://www.trulia.com/for_sale/West_Lafayette,IN/{}/{}/{}/{}/{}/{}/{}".format(num_beds,num_baths,price_range,min_size,house_type,max_age,sale_type),
        "https://www.trulia.com/for_sale/Westfield,IN/{}/{}/{}/{}/{}/{}/{}".format(num_beds,num_baths,price_range,min_size,house_type,max_age,sale_type),
        "https://www.trulia.com/for_sale/Zionsville,IN/{}/{}/{}/{}/{}/{}/{}".format(num_beds,num_baths,price_range,min_size,house_type,max_age,sale_type),
        "https://www.trulia.com/for_sale/Frankfort,IN/{}/{}/{}/{}/{}/{}/{}".format(num_beds,num_baths,price_range,min_size,house_type,max_age,sale_type),
        "https://www.trulia.com/for_sale/Lebanon,IN/{}/{}/{}/{}/{}/{}/{}".format(num_beds,num_baths,price_range,min_size,house_type,max_age,sale_type),
        "https://www.trulia.com/for_sale/Crawfordsville,IN/{}/{}/{}/{}/{}/{}/{}".format(num_beds,num_baths,price_range,min_size,house_type,max_age,sale_type)]

#Print statements can be uncommented for diagnostic purposes.  This section scrapes data from "cards" on Trulia's website.
for x in urls:
    count=1
    y=x
    while(count < 5):  # will go till 4 pages
        print(x)
        req = Request(x, headers=get_headers())  #req all headers
        htmlfile = urlopen(req)
        htmltext = htmlfile.read()
        #print (htmltext)
        soup = BS(htmltext,'html.parser')
        #print (soup.prettify())
       
        for tag in soup.findAll('div',attrs={'data-testid':'property-price'}): #gets price
                row = tag.get_text()
                if not row:
                    row="NA"
#                 print(row)
                price.append(row)

        #for tag in soup.findAll('div',attrs={'class':'Text__TextBase-sc-1i9uasc-0-div Text__TextContainerBase-sc-1i9uasc-1 lcNNgu'}): #gets add
                #row = tag.get_text()
                #print(row)
                #address.append(row)
               
        for tag in soup.findAll('div',attrs={'data-testid':'property-region'}): #add1
                row = tag.get_text()
                if not row:
                    row="NA"
#                 print(row)
                add1.append(row)
               
        for tag in soup.findAll('div',attrs={'data-testid':'property-street'}): #area code
                row = tag.get_text()
                if not row:
                    row="NA"
#                 print(row)
                area.append(row)
               
        for tag in soup.findAll('div',attrs={'data-testid':'property-beds'}): #bed
                row = tag.get_text()
                if not row:
                    row="NA"
#                 print(row)
                bed.append(row)
               
        for tag in soup.findAll('div',attrs={'data-testid':'property-baths'}): #bath
                row = tag.get_text()
                if not row:
                    row="NA"
#                 print(row)
                bath.append(row)
               
        for tag in soup.findAll('div',attrs={'data-testid':'property-floorSpace'}):  #floorsize
                row = tag.get_text()
                if not row:
                    row="NA"
#                 print(row)
                floor.append(row)
            
            # add more code here
        count=count+1
        page=str(count)+"_p"  # changes page,will go till page 4,total 120 links per city
        x=y+page
        
#Print statements to verify that the data has been loaded successfully into the lists.
print(add1[0:10])
print(area[0:10])
print(price[0:10])
print(bed[0:10])
print(bath[0:10])
print(floor[0:10])
data_frame = pd.DataFrame(list(zip(add1,area,price,bed,bath,floor)),columns=["Address","Location","Price","Bed","Bath","Size"]) #Create the dataframe by zipping the lists
data_frame.tail() #Check tail to ensure data is correct, and show how many houses were scraped in total.

https://www.trulia.com/for_sale/Lafayette,IN/3p_beds/2p_baths/100000-320000_price/1750p_sqft/SINGLE-FAMILY_HOME_type/1979p_built/fsbo,resale_lt/
https://www.trulia.com/for_sale/Lafayette,IN/3p_beds/2p_baths/100000-320000_price/1750p_sqft/SINGLE-FAMILY_HOME_type/1979p_built/fsbo,resale_lt/2_p
https://www.trulia.com/for_sale/Lafayette,IN/3p_beds/2p_baths/100000-320000_price/1750p_sqft/SINGLE-FAMILY_HOME_type/1979p_built/fsbo,resale_lt/3_p
https://www.trulia.com/for_sale/Lafayette,IN/3p_beds/2p_baths/100000-320000_price/1750p_sqft/SINGLE-FAMILY_HOME_type/1979p_built/fsbo,resale_lt/4_p
https://www.trulia.com/for_sale/West_Lafayette,IN/3p_beds/2p_baths/100000-320000_price/1750p_sqft/SINGLE-FAMILY_HOME_type/1979p_built/fsbo,resale_lt/
https://www.trulia.com/for_sale/West_Lafayette,IN/3p_beds/2p_baths/100000-320000_price/1750p_sqft/SINGLE-FAMILY_HOME_type/1979p_built/fsbo,resale_lt/2_p
https://www.trulia.com/for_sale/West_Lafayette,IN/3p_beds/2p_baths/100000-320000_price/1750p_sqft/SINGLE-FAM

,Address,Location,Price,Bed,Bath,Size
223,"Thorntown, IN",123 N Elm St,"$197,000",3bd,2ba,"1,800 sqft"
224,"Ladoga, IN",5526 E 1200 S,"$120,000",4bd,2ba,"1,808 sqft"
225,"Crawfordsville, IN",307 Diamond Ln,"$225,000",3bd,2ba,"1,921 sqft"
226,"Thorntown, IN",123 N Elm St,"$197,000",3bd,2ba,"1,800 sqft"
227,"Ladoga, IN",5526 E 1200 S,"$120,000",4bd,2ba,"1,808 sqft"


In [3]:
#Save the obtained dataframe to csv
data_frame.to_csv('new_Indiana_home.csv') #Export the data to a csv file

### We can load the data from a csv file for developing the code, although when we run the full program we will pull fresh data from Trulia.

In [4]:
#import csv
import pandas as pd
import re
df = pd.read_csv('new_Indiana_home.csv',index_col=[0]) #Create a dataframe using the Trulia data that we scraped
df.head(5) #Verify that the data has been loaded successfully and correctly

,Address,Location,Price,Bed,Bath,Size
0,"Lafayette, IN",4118 Penny Packers Mill Rd,"$225,000",3bd,3ba,"2,014 sqft"
1,"Jesco Hills, Lafayette, IN",2519 Rainbow Dr,"$319,000",4bd,4ba,"3,437 sqft"
2,"Lafayette, IN",1423 Sourgum Ln,"$207,000",3bd,3ba,"1,996 sqft"
3,"West Lafayette, IN",1882 Ellison Dr,"$189,900",3bd,3ba,"1,824 sqft"
4,"West Lafayette, IN",124 S 875 W,"$285,000",3bd,3ba,"2,000 sqft"


In [5]:
#Pre-processing of data - Stage 1
#We need to convert strings into floats so that we can perform mathematical comparisons later.
df = df[df['Price'] != 'Contact For Price'].reset_index(drop=True)
df['Price'] = df['Price'].map(lambda x: x.lstrip('$').rstrip('+')).str.replace(",","").astype(float)
df['Bed'] = df['Bed'].map(lambda x: x.lstrip('bd').rstrip('bds')).astype(float)
df['Bath'] = df['Bath'].map(lambda x: x.lstrip('ba').rstrip('AaBbCc')).astype(float)
df['Size'] = df['Size'].map(lambda x: x.lstrip('sf').rstrip('sqft')).str.replace(",","").astype(float)
df2 = df["Address"].apply(lambda x: x.split(","))
df['City'] = df2.apply(lambda x: " ".join(x[-2].split()[:])) #Create a new city column in case we want to group by city later.

In [6]:
#Verify that pre-processing went as expected and datatypes make sense.
print(df.head())
print(df.dtypes.value_counts())

                      Address                    Location     Price  Bed  \
0               Lafayette, IN  4118 Penny Packers Mill Rd  225000.0  3.0   
1  Jesco Hills, Lafayette, IN             2519 Rainbow Dr  319000.0  4.0   
2               Lafayette, IN             1423 Sourgum Ln  207000.0  3.0   
3          West Lafayette, IN             1882 Ellison Dr  189900.0  3.0   
4          West Lafayette, IN                 124 S 875 W  285000.0  3.0   

   Bath    Size            City  
0   3.0  2014.0       Lafayette  
1   4.0  3437.0       Lafayette  
2   3.0  1996.0       Lafayette  
3   3.0  1824.0  West Lafayette  
4   3.0  2000.0  West Lafayette  
float64    4
object     3
dtype: int64


In [7]:
#Checking missing values 
df.isnull().sum()

Address     0
Location    0
Price       0
Bed         0
Bath        0
Size        0
City        0
dtype: int64

In [8]:
#Checkng duplicate rows
print("Starting shape is ",df.shape) #get the initial size of the dataframe
# df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True) #This column was introduced when loading in the csv file and should be removed.
# df['dupes'] = df.duplicated(subset=['Address','Price','Bed','Bath','Size'])
df['dupes'] = df.duplicated(subset=['Address','Location']) #Since each house should have a unique street address, we can use this value to remove duplicate listings.
df=df[df['dupes'] != True].reset_index(drop=True)
df.shape #The shape should change if duplicates have been removed.

Starting shape is  (228, 7)


(42, 8)

### The above code shows that a considerable portion of our data was duplicated.  We can make a separate csv file and verify unique entries if we would like.

# Now we need to associate latitude and longitude data with the addresses.

In [11]:
#Import libraries for obtaining GPS coordinates of house addresses and visualizing locations.
import pandas as pd
import numpy as np
# import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
!pip install folium
import folium
from folium.plugins import FastMarkerCluster

     |████████████████████████████████| 93 kB 2.7 MB/s  eta 0:00:01


In [12]:
{
    "tags": [
        "hide-input",
    ]
}
from geopy.geocoders import Bing
bing_key = 'AhZHj8FQAx0m0ZcVnkpgKxTRiUw--XNYwdNys48WdJWgjis8x8dJ62NSP1heq78N'

In [13]:
#test geocoding for a single address.  We will use bing, since Nominatum does not have robust residential data.

locator = Bing(api_key = bing_key, user_agent = 'housing_explorer')
address = df.iloc[3,1] + " " + df.iloc[3,0] + ", United States"
location = locator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))


The geograpical coordinate of 1882 Ellison Dr West Lafayette, IN, United States are 40.470912, -86.940665.


In [14]:
#Create a new column with the full address, which we will feed into our geocoder.
df['full_address'] = df['Location'] + " " + df['Address'] + ", United States"
df.head()

,Address,Location,Price,Bed,Bath,Size,City,dupes,full_address
0,"Lafayette, IN",4118 Penny Packers Mill Rd,225000.0,3.0,3.0,2014.0,Lafayette,False,"4118 Penny Packers Mill Rd Lafayette, IN, Unit..."
1,"Jesco Hills, Lafayette, IN",2519 Rainbow Dr,319000.0,4.0,4.0,3437.0,Lafayette,False,"2519 Rainbow Dr Jesco Hills, Lafayette, IN, Un..."
2,"Lafayette, IN",1423 Sourgum Ln,207000.0,3.0,3.0,1996.0,Lafayette,False,"1423 Sourgum Ln Lafayette, IN, United States"
3,"West Lafayette, IN",1882 Ellison Dr,189900.0,3.0,3.0,1824.0,West Lafayette,False,"1882 Ellison Dr West Lafayette, IN, United States"
4,"West Lafayette, IN",124 S 875 W,285000.0,3.0,3.0,2000.0,West Lafayette,False,"124 S 875 W West Lafayette, IN, United States"


In [15]:
#Use Bing geocoding to obtain lat and long of addresses in a subset of the dataframe
from geopy.extra.rate_limiter import RateLimiter #Rate limiting may not be necessary, but our data set is small so it isn't a great burden.
#If the data set were large, we would verify Bing's ToS to see if batch calls can be made without delay.
df_test=df.iloc[:5] #create a subset of the dataframe to test code with
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df_test['location'] = df_test['full_address'].apply(geocode)
df_test['point'] = df_test['location'].apply(lambda loc: tuple(loc.point) if loc else None)
df_test[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df_test['point'].tolist(), index=df_test.index)
df_test.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

,Address,Location,Price,Bed,Bath,Size,City,dupes,full_address,location,point,latitude,longitude,altitude
0,"Lafayette, IN",4118 Penny Packers Mill Rd,225000.0,3.0,3.0,2014.0,Lafayette,False,"4118 Penny Packers Mill Rd Lafayette, IN, Unit...","(4118 Penny Packers Mill Rd, Lafayette, IN 479...","(40.356407, -86.881485, 0.0)",40.356407,-86.881485,0.0
1,"Jesco Hills, Lafayette, IN",2519 Rainbow Dr,319000.0,4.0,4.0,3437.0,Lafayette,False,"2519 Rainbow Dr Jesco Hills, Lafayette, IN, Un...","(2519 Rainbow Dr, Lafayette, IN 47904, United ...","(40.428314, -86.865967, 0.0)",40.428314,-86.865967,0.0
2,"Lafayette, IN",1423 Sourgum Ln,207000.0,3.0,3.0,1996.0,Lafayette,False,"1423 Sourgum Ln Lafayette, IN, United States","(1423 Sourgum Ln, Lafayette, IN 47905, United ...","(40.395855, -86.829083, 0.0)",40.395855,-86.829083,0.0
3,"West Lafayette, IN",1882 Ellison Dr,189900.0,3.0,3.0,1824.0,West Lafayette,False,"1882 Ellison Dr West Lafayette, IN, United States","(1882 Ellison Dr, West Lafayette, IN 47906, Un...","(40.470912, -86.940665, 0.0)",40.470912,-86.940665,0.0
4,"West Lafayette, IN",124 S 875 W,285000.0,3.0,3.0,2000.0,West Lafayette,False,"124 S 875 W West Lafayette, IN, United States","(124 S 875 W, West Lafayette, IN 47906, United...","(40.414595, -87.069837, 0.0)",40.414595,-87.069837,0.0


In [16]:
#Since test worked, run on the full dataframe.
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df['location'] = df['full_address'].apply(geocode)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)
df.head(25)

,Address,Location,Price,Bed,Bath,Size,City,dupes,full_address,location,point,latitude,longitude,altitude
0,"Lafayette, IN",4118 Penny Packers Mill Rd,225000.0,3.0,3.0,2014.0,Lafayette,False,"4118 Penny Packers Mill Rd Lafayette, IN, Unit...","(4118 Penny Packers Mill Rd, Lafayette, IN 479...","(40.356407, -86.881485, 0.0)",40.356407,-86.881485,0.0
1,"Jesco Hills, Lafayette, IN",2519 Rainbow Dr,319000.0,4.0,4.0,3437.0,Lafayette,False,"2519 Rainbow Dr Jesco Hills, Lafayette, IN, Un...","(2519 Rainbow Dr, Lafayette, IN 47904, United ...","(40.428314, -86.865967, 0.0)",40.428314,-86.865967,0.0
2,"Lafayette, IN",1423 Sourgum Ln,207000.0,3.0,3.0,1996.0,Lafayette,False,"1423 Sourgum Ln Lafayette, IN, United States","(1423 Sourgum Ln, Lafayette, IN 47905, United ...","(40.395855, -86.829083, 0.0)",40.395855,-86.829083,0.0
3,"West Lafayette, IN",1882 Ellison Dr,189900.0,3.0,3.0,1824.0,West Lafayette,False,"1882 Ellison Dr West Lafayette, IN, United States","(1882 Ellison Dr, West Lafayette, IN 47906, Un...","(40.470912, -86.940665, 0.0)",40.470912,-86.940665,0.0
4,"West Lafayette, IN",124 S 875 W,285000.0,3.0,3.0,2000.0,West Lafayette,False,"124 S 875 W West Lafayette, IN, United States","(124 S 875 W, West Lafayette, IN 47906, United...","(40.414595, -87.069837, 0.0)",40.414595,-87.069837,0.0
5,"Lafayette, IN",4305 Fletcher Dr,214900.0,4.0,3.0,3592.0,Lafayette,False,"4305 Fletcher Dr Lafayette, IN, United States","(4305 Fletcher Dr, Lafayette, IN 47909, United...","(40.355457, -86.847656, 0.0)",40.355457,-86.847656,0.0
6,"West Lafayette, IN",2135 Ringneck Rd,319000.0,4.0,4.0,2432.0,West Lafayette,False,"2135 Ringneck Rd West Lafayette, IN, United St...","(2135 Ringneck Rd, West Lafayette, IN 47906, U...","(40.44897, -86.959076, 0.0)",40.448970,-86.959076,0.0
7,"Lafayette, IN",2101 Fieldstone Dr,314900.0,4.0,3.0,2276.0,Lafayette,False,"2101 Fieldstone Dr Lafayette, IN, United States","(2101 Fieldstone Dr, Lafayette, IN 47909, Unit...","(40.351856, -86.86454, 0.0)",40.351856,-86.864540,0.0
8,"Lafayette, IN",4030 Fletcher Dr,210000.0,3.0,3.0,3036.0,Lafayette,False,"4030 Fletcher Dr Lafayette, IN, United States","(4030 Fletcher Dr, Lafayette, IN 47909, United...","(40.359222, -86.848447, 0.0)",40.359222,-86.848447,0.0
9,"Lafayette, IN",2604 Margesson Xing,177000.0,3.0,3.0,2400.0,Lafayette,False,"2604 Margesson Xing Lafayette, IN, United States","(2604 Margesson Crossing, Lafayette, IN 47909,...","(40.356659, -86.855088, 0.0)",40.356659,-86.855088,0.0


In [17]:
#Clean up the dataframe a bit by removing unnecessary columns.
df_homes = df.drop(['location','point','altitude','dupes'],axis=1)
df_homes.reset_index(inplace=True)
df_homes.head()

,index,Address,Location,Price,Bed,Bath,Size,City,full_address,latitude,longitude
0,0,"Lafayette, IN",4118 Penny Packers Mill Rd,225000.0,3.0,3.0,2014.0,Lafayette,"4118 Penny Packers Mill Rd Lafayette, IN, Unit...",40.356407,-86.881485
1,1,"Jesco Hills, Lafayette, IN",2519 Rainbow Dr,319000.0,4.0,4.0,3437.0,Lafayette,"2519 Rainbow Dr Jesco Hills, Lafayette, IN, Un...",40.428314,-86.865967
2,2,"Lafayette, IN",1423 Sourgum Ln,207000.0,3.0,3.0,1996.0,Lafayette,"1423 Sourgum Ln Lafayette, IN, United States",40.395855,-86.829083
3,3,"West Lafayette, IN",1882 Ellison Dr,189900.0,3.0,3.0,1824.0,West Lafayette,"1882 Ellison Dr West Lafayette, IN, United States",40.470912,-86.940665
4,4,"West Lafayette, IN",124 S 875 W,285000.0,3.0,3.0,2000.0,West Lafayette,"124 S 875 W West Lafayette, IN, United States",40.414595,-87.069837


In [18]:
#Create a map to visualize house locations
map_indiana = folium.Map(location=[40.166,-86.557], zoom_start=10)

# add markers to map for each house in dataset.
for lat, lng, add, price in zip(df['latitude'], df['longitude'], df['Location'], df['Price']):
    label = '{}, {}'.format(price, add)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_indiana)  

map_indiana

# Visualization shows that most of our addresses are correct, but some may appear in obviously incorrect locations.  We should probably make some code to eliminate homes with these incorrect coordinates.

In [19]:
#I know approximately the region that I am interested in.  Therefore, I can set maximum and minimum latitude and longitude values, and check that the points fall within these ranges.
#Any coordinates outside this zone will be eliminated.
#Create acceptable latitude and longitude threshholds
lat_max = 40.73
lat_min = 39.88
lng_max = -85.87
lng_min = -87.17

#Convert latitude and longitude values to float type in dataframe
df_homes['latitude'] = df_homes['latitude'].astype(float)
df_homes['longitude'] = df_homes['longitude'].astype(float)

print("Starting shape is ", df_homes.shape)
df_homes['lat_check'] = df_homes['latitude'].apply(lambda x: x < lat_min or x > lat_max)
df_homes = df_homes[df_homes['lat_check'] == False].reset_index(drop=True)
print("Shape after dropping latitude outliers is ", df_homes.shape)
df_homes['lng_check'] = df_homes['longitude'].apply(lambda x: x < lng_min or x > lng_max)
df_homes = df_homes[df_homes['lng_check'] == False].reset_index(drop=True)
print("Shape after dropping longitude outliers is ", df_homes.shape) #Shows how many data points have been dropped for being outside of desired geographical range.

Starting shape is  (42, 11)
Shape after dropping latitude outliers is  (33, 12)
Shape after dropping longitude outliers is  (33, 13)


In [20]:
#Looks like we have removed the incorrect coordinate addresses.  Let's visualize to be sure.
#Create a map to visualize house locations
map_indiana = folium.Map(location=[40.166,-86.557], zoom_start=10)

for lat, lng, add, price in zip(df_homes['latitude'], df_homes['longitude'], df_homes['Location'], df_homes['Price']):
    label = '{}, {}'.format(price, add)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_indiana)  

map_indiana

### Excellent.  There are no longer any markers outside of our designated area of acceptability.

Now it's time to gather some data from Foursquare.  Costco is our favorite store, so we need to know if there are any nearby.

In [21]:
{
    "tags": [
        "hide-input",
    ]
}
#Establish credentials
CLIENT_ID = 'QUFSJQ0MD1TTDMU2MBNKE43MVECHVG4LLCXDUT2KNBBJKEIE' # your Foursquare ID
CLIENT_SECRET = 'HKE3PZU5JXAXHVIOF01B3PF2NOIIJGSEH0N01AEYP1P5CNCT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [22]:
#Search for Costco locations.  I know that there are none in or near Lafayette, so we don't need to search there.
venue_name = 'Costco'
costco_url = 'https://api.foursquare.com/v2/venues/search?near=Zionsville,_IN&query={}&client_id={}&client_secret={}&v={}'.format(venue_name,CLIENT_ID,CLIENT_SECRET, VERSION)
costco_url

'https://api.foursquare.com/v2/venues/search?near=Zionsville,_IN&query=Costco&client_id=QUFSJQ0MD1TTDMU2MBNKE43MVECHVG4LLCXDUT2KNBBJKEIE&client_secret=HKE3PZU5JXAXHVIOF01B3PF2NOIIJGSEH0N01AEYP1P5CNCT&v=20180605'

In [23]:
#Find the locations of nearby Costco stores.
import requests # library to handle requests
results = requests.get(costco_url).json()
print(results)

venues = results['response']['venues']
costco_frame = pd.json_normalize(venues)
costco_locations = costco_frame[['id','name','location.address','location.lat','location.lng']]
# costco_locations[0:200]

{'meta': {'code': 200, 'requestId': '5fea585ff26a273600606e46'}, 'response': {'venues': [{'id': '4ae33851f964a520dc9121e3', 'name': 'Costco', 'location': {'address': '9010 Michigan Rd', 'crossStreet': 'btwn Vincennes Rd & W 92nd St', 'lat': 39.917311923791274, 'lng': -86.22762019938833, 'labeledLatLngs': [{'label': 'display', 'lat': 39.917311923791274, 'lng': -86.22762019938833}], 'postalCode': '46268', 'cc': 'US', 'city': 'Indianapolis', 'state': 'IN', 'country': 'United States', 'formattedAddress': ['9010 Michigan Rd (btwn Vincennes Rd & W 92nd St)', 'Indianapolis, IN 46268', 'United States']}, 'categories': [{'id': '52e816a6bcbc57f1066b7a54', 'name': 'Warehouse Store', 'pluralName': 'Warehouse Stores', 'shortName': 'Warehouse Store', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/default_', 'suffix': '.png'}, 'primary': True}], 'referralId': 'v-1609193567', 'hasPerk': False}, {'id': '4db5e39c5da3a76f445b3620', 'name': 'Costco Gasoline', 'location': {'address': '901

In [24]:
#We are only interested in the store, and not auxiliary features such as pharmacy or gas station, so filter the dataframe accordingly.
costco_locations = costco_locations[costco_locations['name'] == 'Costco'].reset_index(drop=True).dropna()
costco_locations

,id,name,location.address,location.lat,location.lng
0,4ae33851f964a520dc9121e3,Costco,9010 Michigan Rd,39.917312,-86.227620
1,4b367462f964a520d23525e3,Costco,6110 E 86th St,39.914035,-86.062033
2,576c2c6b498e8fac2bf2bc8d,Costco,4628 E County Line Rd,39.638273,-86.089478


In [25]:
#Now we want to calculate the distance between the house and the nearest costco
from geopy import distance
house_loc = (df_homes.latitude[10],df_homes.longitude[10])
print(house_loc)
costco_loc = (costco_locations.loc[0,'location.lat'],costco_locations.loc[0,'location.lng'])
print(costco_loc)
print(distance.distance(house_loc,costco_loc).miles)

(40.355357, -86.833444)
(39.917311923791274, -86.22762019938833)
44.07555125086898


### Now that we know our distance calculation works, we create a function to perform the task on all addresses in the dataframe.

In [26]:
#Time to make a function to calculate the distance from the house to each costco, then store the shortest distance.
def get_costco_distance(h_lats,h_lngs,c_lats,c_lngs):
    distance_list =[] #create empty list to store distances
    
    for h_lat,h_lng in zip(h_lats,h_lngs): #loop through the latitude and longitude of the houses
        h_gps = (h_lat,h_lng) #create a variable to hold gps coordinates
#         print("GPS units of house are: ", h_gps)
        min_distance = 500 #set an arbitrarily high distance to costco
        
        for c_lat,c_lng in zip(c_lats,c_lngs):#loop through the latitude and longitude of costco locations
            c_gps = (c_lat,c_lng)
#             print("GPS units of costco are: ", c_gps)
            temp_distance = distance.distance(h_gps,c_gps).miles
#             print("Temp distance is: ", temp_distance)
            if temp_distance < min_distance:
                min_distance = temp_distance
#                 print('New minimum distance is: ',min_distance)
        distance_list.append(min_distance)
    print(distance_list)
    return distance_list

In [27]:
#Call the function
from geopy import distance
distances = get_costco_distance(df_homes['latitude'],df_homes['longitude'],costco_locations['location.lat'],costco_locations['location.lng'])

[46.007856459359395, 48.83152408451617, 45.86934524748339, 53.68695571124422, 56.25469610307553, 44.630985027925725, 53.32786023611768, 45.12801152586063, 44.83738299820899, 44.97694599003779, 44.07555125086898, 52.350186223317834, 52.64136804866537, 53.3313336326996, 50.831585582118926, 72.86166776316016, 62.537560369388125, 62.421287793934106, 9.714610243228913, 6.992622945323302, 8.82415477984302, 9.229864375754465, 9.63719357392964, 9.454387328143865, 7.977414748083444, 25.13560789451191, 38.71426426719332, 36.99571291208435, 39.89357676094572, 40.354050226636986, 39.89351682522584, 36.63785729349072, 34.699131064177436]


### We will also calculate distance to family members.

In [28]:
#Create GPS locations for family.  For privacy reasons, these are arbitrary locations to show proof of concept.
# df_cleaned['GPS']=(df_cleaned['latitude'],df_cleaned['longitude'])
# df_cleaned.head()
family_dist_1 = (40.36,-86.86)
family_dist_2 = (40.07,-86.48)
family_dist_3 = (40.01,-86.13)

In [29]:
#Time to make a function to calculate the distance from the house to each costco, then store the shortest distance.
def get_family_distance(h_lats,h_lngs,f_gps):
    fam_distance_list =[] #create empty list to store distances
    
    for h_lat,h_lng in zip(h_lats,h_lngs): #loop through the latitude and longitude of the houses
        h_gps = (h_lat,h_lng) #create a variable to hold gps coordinates
#         print("GPS units of house are: ", h_gps)
        fam_dist = distance.distance(h_gps,f_gps).miles
#             print("Temp distance is: ", temp_distance)
#             print('New minimum distance is: ',min_distance)
        fam_distance_list.append(fam_dist)
    print(fam_distance_list)
    return fam_distance_list

In [30]:
#Calculate distances from family members for each house.
from geopy import distance
grandparent_distance = get_family_distance(df_homes['latitude'],df_homes['longitude'],family_dist_1)
older_distance = get_family_distance(df_homes['latitude'],df_homes['longitude'],family_dist_3)
younger_distance = get_family_distance(df_homes['latitude'],df_homes['longitude'],family_dist_2)

[1.1608236340156461, 4.724058728076005, 2.9634197502572666, 8.755725860003851, 11.694371682853772, 0.7230375812370485, 8.061973359616161, 0.6108864268791456, 0.6121473604600466, 0.34693117716619415, 1.4378582256687786, 7.828002672217391, 8.655257747156048, 8.969131362671577, 19.321514532574074, 28.38684500738414, 26.092290452264603, 25.952695603451236, 41.816878884830125, 46.387797624017665, 45.649291578829505, 46.47359320122452, 46.044859754675045, 49.820048986640344, 37.47777370721703, 20.571534664073837, 35.98658100263408, 37.46018506528272, 37.39518543810147, 34.46174957802251, 37.2769814218674, 37.60778122119001, 21.923830423144675]
[46.39730926358706, 48.45868099149407, 45.569351451959314, 53.37205421670634, 57.01509736684011, 44.83774800536468, 53.28786836443641, 45.468501018914054, 45.010840206683895, 45.214881092129744, 44.19884109431517, 51.90095654055234, 51.99255625097661, 52.776740586121534, 47.82292965401496, 71.82552702473974, 59.55487733812405, 59.44935360650045, 3.9230

In [31]:
#Now we add the data to the dataframe.
df_homes['dist_to_costco'] = distances
df_homes['grand_dist'] = grandparent_distance
df_homes['older_dist'] = older_distance
df_homes['younger_dist'] = younger_distance
df_homes.head()

,index,Address,Location,Price,Bed,Bath,Size,City,full_address,latitude,longitude,lat_check,lng_check,dist_to_costco,grand_dist,older_dist,younger_dist
0,0,"Lafayette, IN",4118 Penny Packers Mill Rd,225000.0,3.0,3.0,2014.0,Lafayette,"4118 Penny Packers Mill Rd Lafayette, IN, Unit...",40.356407,-86.881485,False,False,46.007856,1.160824,46.397309,29.008622
1,1,"Jesco Hills, Lafayette, IN",2519 Rainbow Dr,319000.0,4.0,4.0,3437.0,Lafayette,"2519 Rainbow Dr Jesco Hills, Lafayette, IN, Un...",40.428314,-86.865967,False,False,48.831524,4.724059,48.458681,32.055756
2,2,"Lafayette, IN",1423 Sourgum Ln,207000.0,3.0,3.0,1996.0,Lafayette,"1423 Sourgum Ln Lafayette, IN, United States",40.395855,-86.829083,False,False,45.869345,2.963420,45.569351,29.090137
3,3,"West Lafayette, IN",1882 Ellison Dr,189900.0,3.0,3.0,1824.0,West Lafayette,"1882 Ellison Dr West Lafayette, IN, United States",40.470912,-86.940665,False,False,53.686956,8.755726,53.372054,36.850050
4,4,"West Lafayette, IN",124 S 875 W,285000.0,3.0,3.0,2000.0,West Lafayette,"124 S 875 W West Lafayette, IN, United States",40.414595,-87.069837,False,False,56.254696,11.694372,57.015097,39.215871


In [32]:
#Remove unnecessary columns and save to a new csv file.
df_homes = df_homes.drop(['lat_check','lng_check'],axis=1)
df_homes.to_csv('cleaned_homes.csv')

In [33]:
#import csv
import pandas as pd
import re
df_cleaned = pd.read_csv('cleaned_homes.csv',index_col = [0]) #Create a dataframe using the Trulia data that we scraped
df_cleaned.head(5) #Verify that the data has been loaded successfully and correctly

,index,Address,Location,Price,Bed,Bath,Size,City,full_address,latitude,longitude,dist_to_costco,grand_dist,older_dist,younger_dist
0,0,"Lafayette, IN",4118 Penny Packers Mill Rd,225000.0,3.0,3.0,2014.0,Lafayette,"4118 Penny Packers Mill Rd Lafayette, IN, Unit...",40.356407,-86.881485,46.007856,1.160824,46.397309,29.008622
1,1,"Jesco Hills, Lafayette, IN",2519 Rainbow Dr,319000.0,4.0,4.0,3437.0,Lafayette,"2519 Rainbow Dr Jesco Hills, Lafayette, IN, Un...",40.428314,-86.865967,48.831524,4.724059,48.458681,32.055756
2,2,"Lafayette, IN",1423 Sourgum Ln,207000.0,3.0,3.0,1996.0,Lafayette,"1423 Sourgum Ln Lafayette, IN, United States",40.395855,-86.829083,45.869345,2.963420,45.569351,29.090137
3,3,"West Lafayette, IN",1882 Ellison Dr,189900.0,3.0,3.0,1824.0,West Lafayette,"1882 Ellison Dr West Lafayette, IN, United States",40.470912,-86.940665,53.686956,8.755726,53.372054,36.850050
4,4,"West Lafayette, IN",124 S 875 W,285000.0,3.0,3.0,2000.0,West Lafayette,"124 S 875 W West Lafayette, IN, United States",40.414595,-87.069837,56.254696,11.694372,57.015097,39.215871


In [34]:
#Test to verify that we can discover nearby venues using Foursquare API
import requests
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, df_cleaned['latitude'][0], df_cleaned['longitude'][0], VERSION, radius, LIMIT)
url
results_test = requests.get(url).json()
results_test

{'meta': {'code': 200, 'requestId': '5fea5886f26a27360060eb6d'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Lafayette',
  'headerFullLocation': 'Lafayette',
  'headerLocationGranularity': 'city',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 40.3609070045, 'lng': -86.8755907499273},
   'sw': {'lat': 40.351906995499995, 'lng': -86.8873792500727}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e9f5800be7b667c5f8a0087',
       'name': "O'steeles Gym",
       'location': {'lat': 40.358634,
        'lng': -86.882035,
        'labeledLatLngs': [{'label': 'display',
          'lat': 40.358634,
          'lng': -86.882035}],
        'di

In [35]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [36]:
#Function for searching nearby venues of each home's address.
def getNearbyVenues(names, latitudes, longitudes):#, radius=8000): #Radius is equivalent to 5 miles.
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        #url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            #radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Home Address', 
                  'Home Latitude', 
                  'Home Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [37]:
#Retrieve nearby venues for each home's address.
home_venues = getNearbyVenues(names = df_cleaned['full_address'], latitudes = df_cleaned['latitude'],longitudes = df_cleaned['longitude'])

4118 Penny Packers Mill Rd Lafayette, IN, United States
2519 Rainbow Dr Jesco Hills, Lafayette, IN, United States
1423 Sourgum Ln Lafayette, IN, United States
1882 Ellison Dr West Lafayette, IN, United States
124 S 875 W West Lafayette, IN, United States
4305 Fletcher Dr Lafayette, IN, United States
2135 Ringneck Rd West Lafayette, IN, United States
2101 Fieldstone Dr Lafayette, IN, United States
4030 Fletcher Dr Lafayette, IN, United States
2604 Margesson Xing Lafayette, IN, United States
4277 Tripoli Dr Lafayette, IN, United States
3377 Morgan St University Farms, West Lafayette, IN, United States
4523 Joshua Ct Prophet's Ridge, West Lafayette, IN, United States
4517 Hayloft Dr #192 West Lafayette, IN, United States
5990 W 252 N Delphi, IN, United States
7751 N 1000 E Remington, IN, United States
12082 Clearwater Dr W Monticello, IN, United States
11043 Sunset Ct Monticello, IN, United States
18236 Nickel Plate Dr Westfield, IN, United States
2914 Bridlewood Ln Carmel, IN, United Sta

In [38]:
#Check that retrieval worked.
print(home_venues.shape)
home_venues.head()

(2395, 7)


,Home Address,Home Latitude,Home Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"4118 Penny Packers Mill Rd Lafayette, IN, Unit...",40.356407,-86.881485,The End Zone,40.367602,-86.867583,Sports Bar
1,"4118 Penny Packers Mill Rd Lafayette, IN, Unit...",40.356407,-86.881485,Frozen Custard Igloo,40.367688,-86.874390,Ice Cream Shop
2,"4118 Penny Packers Mill Rd Lafayette, IN, Unit...",40.356407,-86.881485,The Pink Walrus,40.365974,-86.862080,Frozen Yogurt Shop
3,"4118 Penny Packers Mill Rd Lafayette, IN, Unit...",40.356407,-86.881485,Arni's,40.367821,-86.861185,Pizza Place
4,"4118 Penny Packers Mill Rd Lafayette, IN, Unit...",40.356407,-86.881485,EMT Food Truck,40.335902,-86.861779,Food Truck


In [39]:
print('There are {} uniques categories.'.format(len(home_venues['Venue Category'].unique())))

There are 172 uniques categories.


In [40]:
#Count unique venues for each unique home address
unique_count=pd.DataFrame(home_venues.groupby(home_venues['Home Address']).nunique())
# unique_count['Venue Category']
df_cleaned.set_index(df_cleaned['full_address'], inplace=True) #set index to match unique_count's index
df_cleaned['unique_count']=unique_count['Venue Category'] #Add count of unique venues to df_cleaned dataframe
df_cleaned.head() #Check that it worked.

,index,Address,Location,Price,Bed,Bath,Size,City,full_address,latitude,longitude,dist_to_costco,grand_dist,older_dist,younger_dist,unique_count
full_address,,,,,,,,,,,,,,,,
"4118 Penny Packers Mill Rd Lafayette, IN, United States",0,"Lafayette, IN",4118 Penny Packers Mill Rd,225000.0,3.0,3.0,2014.0,Lafayette,"4118 Penny Packers Mill Rd Lafayette, IN, Unit...",40.356407,-86.881485,46.007856,1.160824,46.397309,29.008622,55
"2519 Rainbow Dr Jesco Hills, Lafayette, IN, United States",1,"Jesco Hills, Lafayette, IN",2519 Rainbow Dr,319000.0,4.0,4.0,3437.0,Lafayette,"2519 Rainbow Dr Jesco Hills, Lafayette, IN, Un...",40.428314,-86.865967,48.831524,4.724059,48.458681,32.055756,60
"1423 Sourgum Ln Lafayette, IN, United States",2,"Lafayette, IN",1423 Sourgum Ln,207000.0,3.0,3.0,1996.0,Lafayette,"1423 Sourgum Ln Lafayette, IN, United States",40.395855,-86.829083,45.869345,2.963420,45.569351,29.090137,54
"1882 Ellison Dr West Lafayette, IN, United States",3,"West Lafayette, IN",1882 Ellison Dr,189900.0,3.0,3.0,1824.0,West Lafayette,"1882 Ellison Dr West Lafayette, IN, United States",40.470912,-86.940665,53.686956,8.755726,53.372054,36.850050,61
"124 S 875 W West Lafayette, IN, United States",4,"West Lafayette, IN",124 S 875 W,285000.0,3.0,3.0,2000.0,West Lafayette,"124 S 875 W West Lafayette, IN, United States",40.414595,-87.069837,56.254696,11.694372,57.015097,39.215871,12


In [41]:
#Calculate square feet per dollar and add to dataframe
df_cleaned['sqft_per_dollar']=df_cleaned['Size']/df_cleaned['Price']
df_cleaned.head()

,index,Address,Location,Price,Bed,Bath,Size,City,full_address,latitude,longitude,dist_to_costco,grand_dist,older_dist,younger_dist,unique_count,sqft_per_dollar
full_address,,,,,,,,,,,,,,,,,
"4118 Penny Packers Mill Rd Lafayette, IN, United States",0,"Lafayette, IN",4118 Penny Packers Mill Rd,225000.0,3.0,3.0,2014.0,Lafayette,"4118 Penny Packers Mill Rd Lafayette, IN, Unit...",40.356407,-86.881485,46.007856,1.160824,46.397309,29.008622,55,0.008951
"2519 Rainbow Dr Jesco Hills, Lafayette, IN, United States",1,"Jesco Hills, Lafayette, IN",2519 Rainbow Dr,319000.0,4.0,4.0,3437.0,Lafayette,"2519 Rainbow Dr Jesco Hills, Lafayette, IN, Un...",40.428314,-86.865967,48.831524,4.724059,48.458681,32.055756,60,0.010774
"1423 Sourgum Ln Lafayette, IN, United States",2,"Lafayette, IN",1423 Sourgum Ln,207000.0,3.0,3.0,1996.0,Lafayette,"1423 Sourgum Ln Lafayette, IN, United States",40.395855,-86.829083,45.869345,2.963420,45.569351,29.090137,54,0.009643
"1882 Ellison Dr West Lafayette, IN, United States",3,"West Lafayette, IN",1882 Ellison Dr,189900.0,3.0,3.0,1824.0,West Lafayette,"1882 Ellison Dr West Lafayette, IN, United States",40.470912,-86.940665,53.686956,8.755726,53.372054,36.850050,61,0.009605
"124 S 875 W West Lafayette, IN, United States",4,"West Lafayette, IN",124 S 875 W,285000.0,3.0,3.0,2000.0,West Lafayette,"124 S 875 W West Lafayette, IN, United States",40.414595,-87.069837,56.254696,11.694372,57.015097,39.215871,12,0.007018


In [42]:
#Create a weighted distance score.  It is most important that our children live near their grandparents, so we will weight their distance more heavily.
g_weight = 0.5
y_weight = 0.25
o_weight=0.25
df_cleaned['weighted_dist_score']=df_cleaned['grand_dist']*g_weight+df_cleaned['younger_dist']*y_weight+df_cleaned['older_dist']*o_weight
df_cleaned.head()

,index,Address,Location,Price,Bed,Bath,Size,City,full_address,latitude,longitude,dist_to_costco,grand_dist,older_dist,younger_dist,unique_count,sqft_per_dollar,weighted_dist_score
full_address,,,,,,,,,,,,,,,,,,
"4118 Penny Packers Mill Rd Lafayette, IN, United States",0,"Lafayette, IN",4118 Penny Packers Mill Rd,225000.0,3.0,3.0,2014.0,Lafayette,"4118 Penny Packers Mill Rd Lafayette, IN, Unit...",40.356407,-86.881485,46.007856,1.160824,46.397309,29.008622,55,0.008951,19.431895
"2519 Rainbow Dr Jesco Hills, Lafayette, IN, United States",1,"Jesco Hills, Lafayette, IN",2519 Rainbow Dr,319000.0,4.0,4.0,3437.0,Lafayette,"2519 Rainbow Dr Jesco Hills, Lafayette, IN, Un...",40.428314,-86.865967,48.831524,4.724059,48.458681,32.055756,60,0.010774,22.490639
"1423 Sourgum Ln Lafayette, IN, United States",2,"Lafayette, IN",1423 Sourgum Ln,207000.0,3.0,3.0,1996.0,Lafayette,"1423 Sourgum Ln Lafayette, IN, United States",40.395855,-86.829083,45.869345,2.963420,45.569351,29.090137,54,0.009643,20.146582
"1882 Ellison Dr West Lafayette, IN, United States",3,"West Lafayette, IN",1882 Ellison Dr,189900.0,3.0,3.0,1824.0,West Lafayette,"1882 Ellison Dr West Lafayette, IN, United States",40.470912,-86.940665,53.686956,8.755726,53.372054,36.850050,61,0.009605,26.933389
"124 S 875 W West Lafayette, IN, United States",4,"West Lafayette, IN",124 S 875 W,285000.0,3.0,3.0,2000.0,West Lafayette,"124 S 875 W West Lafayette, IN, United States",40.414595,-87.069837,56.254696,11.694372,57.015097,39.215871,12,0.007018,29.904928


In [43]:
#Now we want to scale our variables so that they hold equal weight when modeling.  For example, all values for square feet per dollar are much smaller than distance to Costco.
import sklearn
from sklearn import preprocessing
scaler = test=sklearn.preprocessing.MinMaxScaler()
test_set = df_cleaned[['dist_to_costco','weighted_dist_score','unique_count','sqft_per_dollar']]
test_output = scaler.fit(test_set).transform(test_set)
test_output

array([[5.92315155e-01, 3.42956505e-02, 7.54385965e-01, 2.75515561e-01],
       [6.35183055e-01, 1.44602035e-01, 8.42105263e-01, 4.45650720e-01],
       [5.90212328e-01, 6.00691636e-02, 7.36842105e-01, 3.40035457e-01],
       [7.08896461e-01, 3.04819342e-01, 8.59649123e-01, 3.36540079e-01],
       [7.47878966e-01, 4.11980887e-01, 0.00000000e+00, 9.50796576e-02],
       [5.71411992e-01, 3.52518073e-04, 7.54385965e-01, 1.00000000e+00],
       [7.03444804e-01, 2.87497891e-01, 8.77192982e-01, 1.51656322e-01],
       [5.78957668e-01, 8.18898630e-03, 7.19298246e-01, 1.14690120e-01],
       [5.74545451e-01, 1.84551170e-03, 7.54385965e-01, 7.89325366e-01],
       [5.76664246e-01, 0.00000000e+00, 7.71929825e-01, 7.05542859e-01],
       [5.62979597e-01, 2.72195657e-03, 5.78947368e-01, 5.89333627e-02],
       [6.88602110e-01, 2.63362684e-01, 8.59649123e-01, 1.06773596e-01],
       [6.93022728e-01, 2.82566719e-01, 8.42105263e-01, 2.73567224e-01],
       [7.03497535e-01, 3.01043189e-01, 8.59649123e

In [44]:
#convert array to dataframe and name columns
df_test=pd.DataFrame(data=test_output)
df_test.rename(columns={0:'dist_to_costco',1:'weighted_dist_score',2:'unique_count',3:'sqft_per_dollar'},inplace=True)
df_test.head()

,dist_to_costco,weighted_dist_score,unique_count,sqft_per_dollar
0,0.592315,0.034296,0.754386,0.275516
1,0.635183,0.144602,0.842105,0.445651
2,0.590212,0.060069,0.736842,0.340035
3,0.708896,0.304819,0.859649,0.336540
4,0.747879,0.411981,0.000000,0.095080


In [45]:
#Now, we are going to use the values in our df_test dataframe to create an overall score for each house.
#We want the highest overall score to indicate the "best" house.  Therefore:
#A closer distance to Costco and family is desirable, so take 1-scaled distance for each variable
#More squarefeet per dollar is desirable, so take 1-scaled sqft_per_dollar. This gives a score of 1 for highest purchasing power
#More unique venues are desirable, so leave value as is.
#Finally, these variables are not equally important, so assign weights to each.
costco_wt = 1
family_wt = 1.5
unique_wt=0.5
purchase_wt = 1.2

df_test['overall_score']= \
(1-df_test['dist_to_costco'])*costco_wt \
+(1-df_test['weighted_dist_score'])*family_wt \
+df_test['unique_count']*unique_wt \
+(1-df_test['sqft_per_dollar'])*purchase_wt
df_test.head()

,dist_to_costco,weighted_dist_score,unique_count,sqft_per_dollar,overall_score
0,0.592315,0.034296,0.754386,0.275516,3.102816
1,0.635183,0.144602,0.842105,0.445651,2.734186
2,0.590212,0.060069,0.736842,0.340035,2.980062
3,0.708896,0.304819,0.859649,0.336540,2.559851
4,0.747879,0.411981,0.000000,0.095080,2.220054


In [46]:
#Add the overall score to the df_cleaned dataframe.
df_cleaned.set_index(df_cleaned['index'],inplace=True)
print(df_cleaned.head())
df_cleaned['overall_score']=scaler.fit(df_test[['overall_score']]).transform(df_test[['overall_score']])*100
df_cleaned.head()

       index                     Address                    Location  \
index                                                                  
0          0               Lafayette, IN  4118 Penny Packers Mill Rd   
1          1  Jesco Hills, Lafayette, IN             2519 Rainbow Dr   
2          2               Lafayette, IN             1423 Sourgum Ln   
3          3          West Lafayette, IN             1882 Ellison Dr   
4          4          West Lafayette, IN                 124 S 875 W   

          Price  Bed  Bath    Size            City  \
index                                                
0      225000.0  3.0   3.0  2014.0       Lafayette   
1      319000.0  4.0   4.0  3437.0       Lafayette   
2      207000.0  3.0   3.0  1996.0       Lafayette   
3      189900.0  3.0   3.0  1824.0  West Lafayette   
4      285000.0  3.0   3.0  2000.0  West Lafayette   

                                            full_address   latitude  \
index                                        

,index,Address,Location,Price,Bed,Bath,Size,City,full_address,latitude,longitude,dist_to_costco,grand_dist,older_dist,younger_dist,unique_count,sqft_per_dollar,weighted_dist_score,overall_score
index,,,,,,,,,,,,,,,,,,,
0,0,"Lafayette, IN",4118 Penny Packers Mill Rd,225000.0,3.0,3.0,2014.0,Lafayette,"4118 Penny Packers Mill Rd Lafayette, IN, Unit...",40.356407,-86.881485,46.007856,1.160824,46.397309,29.008622,55,0.008951,19.431895,78.647934
1,1,"Jesco Hills, Lafayette, IN",2519 Rainbow Dr,319000.0,4.0,4.0,3437.0,Lafayette,"2519 Rainbow Dr Jesco Hills, Lafayette, IN, Un...",40.428314,-86.865967,48.831524,4.724059,48.458681,32.055756,60,0.010774,22.490639,63.280559
2,2,"Lafayette, IN",1423 Sourgum Ln,207000.0,3.0,3.0,1996.0,Lafayette,"1423 Sourgum Ln Lafayette, IN, United States",40.395855,-86.829083,45.869345,2.963420,45.569351,29.090137,54,0.009643,20.146582,73.530621
3,3,"West Lafayette, IN",1882 Ellison Dr,189900.0,3.0,3.0,1824.0,West Lafayette,"1882 Ellison Dr West Lafayette, IN, United States",40.470912,-86.940665,53.686956,8.755726,53.372054,36.850050,61,0.009605,26.933389,56.012930
4,4,"West Lafayette, IN",124 S 875 W,285000.0,3.0,3.0,2000.0,West Lafayette,"124 S 875 W West Lafayette, IN, United States",40.414595,-87.069837,56.254696,11.694372,57.015097,39.215871,12,0.007018,29.904928,41.847545


In [47]:
#Now we can map the houses!
#Create an empty map instance at appropriate location and zoom
import folium
map_indiana = folium.Map(location=[40.166,-86.557], zoom_start=10)

#Add marker to the map for each house.
for lat, lng, add, score in zip(df_cleaned['latitude'], df_cleaned['longitude'], df_cleaned['Location'], df_cleaned['overall_score']):
    label = 'Overall Score: {}, Address: {}'.format(score, add)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_indiana)  

map_indiana

In [51]:
df_final=df_cleaned[['full_address','Price','Size','overall_score']]
df_final.sort_values('overall_score', ascending=False, inplace=True)
df_final.head(10)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,full_address,Price,Size,overall_score
index,,,,
27,"2914 Bridlewood Ln Carmel, IN, United States",298000.0,1848.0,100.000000
26,"18236 Nickel Plate Dr Westfield, IN, United St...",299900.0,2289.0,96.852326
28,"3459 Heathcliff Ct Westfield, IN, United States",279900.0,2326.0,90.150767
10,"4277 Tripoli Dr Lafayette, IN, United States",319900.0,2121.0,89.023004
7,"2101 Fieldstone Dr Lafayette, IN, United States",314900.0,2276.0,88.151258
30,"4814 Ashbrook Dr Noblesville, IN, United States",309999.0,2615.0,86.174728
32,"5773 Weeping Willow Pl Whitestown, IN, United ...",265000.0,2514.0,84.731763
29,"5202 Ashbrook Dr Noblesville, IN, United States",250000.0,2214.0,83.495584
0,"4118 Penny Packers Mill Rd Lafayette, IN, Unit...",225000.0,2014.0,78.647934


# End project here.